<a href="https://colab.research.google.com/github/EddyGiusepe/Pytorch_Deep_Learning/blob/main/Classification_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h2 align='center'>Classificação binária, Breast Cancer, com Pytorch</h2>


Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Este Script está baseado no seguinte vídeo: [Classificação Binária com Pytorch](https://www.youtube.com/watch?v=cZf3CMdUKIY&t=38s).

## Importação das bibliotecas

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
torch.__version__


'1.10.0+cu111'

## Base de Dados

In [2]:
np.random.seed(123)

torch.manual_seed(123)

In [3]:
Dataset = pd.read_csv('/content/drive/MyDrive/2_DEEP_LEARNING_REDES_NEURAIS_Jorge/1_Pytorch_Deep_Learning/Pytorch_examples/data_breast_cancer.csv')

Dataset.head(6)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN


In [4]:
# Criando a classe

# creating instance of labelencoder
labelencoder = LabelEncoder()

classe = pd.DataFrame(labelencoder.fit_transform(Dataset['diagnosis']))
classe.sample(6)

,0
333,0
273,0
201,1
178,0
85,1
500,0


In [5]:
# criando os previsores

previsores = Dataset.drop(columns=['id', 'diagnosis', 'Unnamed: 32'])
# previsores = Dataset.drop(['id', 'diagnosis', 'Unnamed: 32'], axis=1, inplace=True) --> Assim não funcionou não


In [6]:
previsores.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
previsores.shape

(569, 30)

In [8]:
# Convertemos para TENSORES 
previsores = torch.tensor(np.array(previsores), dtype=torch.float)

classe = torch.tensor(np.array(classe), dtype=torch.float)


In [9]:
# Verificamos o TENSOR
previsores.shape

torch.Size([569, 30])

In [10]:
# Também
type(previsores)

torch.Tensor

In [11]:
previsores

tensor([[1.7990e+01, 1.0380e+01, 1.2280e+02,  ..., 2.6540e-01, 4.6010e-01,
         1.1890e-01],
        [2.0570e+01, 1.7770e+01, 1.3290e+02,  ..., 1.8600e-01, 2.7500e-01,
         8.9020e-02],
        [1.9690e+01, 2.1250e+01, 1.3000e+02,  ..., 2.4300e-01, 3.6130e-01,
         8.7580e-02],
        ...,
        [1.6600e+01, 2.8080e+01, 1.0830e+02,  ..., 1.4180e-01, 2.2180e-01,
         7.8200e-02],
        [2.0600e+01, 2.9330e+01, 1.4010e+02,  ..., 2.6500e-01, 4.0870e-01,
         1.2400e-01],
        [7.7600e+00, 2.4540e+01, 4.7920e+01,  ..., 0.0000e+00, 2.8710e-01,
         7.0390e-02]])

## Transformação dos Dados para Tensores

In [12]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe),
                                           batch_size=10, shuffle=True) 


In [14]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

## Construção do Modelo

In [ ]:
class classificador_torch(nn.Module):
  